In [1]:
!kubectl get ns

NAME              STATUS   AGE
kube-system       Active   8d
kube-public       Active   8d
kube-node-lease   Active   8d
default           Active   8d
kubeflow          Active   8d
istio-system      Active   8d
cert-manager      Active   8d
knative-serving   Active   8d
anonymous         Active   14h


In [89]:
!kubectl delete svc/tfserving-service deployment/tfserving -n tfserving

service "tfserving-service" deleted
deployment.apps "tfserving" deleted


In [46]:
!kubectl create ns tfserving

namespace/tfserving created


In [47]:
!kubectl label ns tfserving serving.kubeflow.org/inferenceservice=enabled

namespace/tfserving labeled


In [90]:
!kubectl apply -f tfserving.yaml -n tfserving

service/tfserving-service created
deployment.apps/tfserving created


In [94]:
!kubectl get pods -n tfserving

NAME                         READY   STATUS    RESTARTS   AGE
tfserving-6955975fff-hn9kl   1/1     Running   0          2m6s


In [93]:
!kubectl describe pods tfserving-6955975fff-hn9kl -n tfserving

Name:         tfserving-6955975fff-hn9kl
Namespace:    tfserving
Priority:     0
Node:         ip-172-16-0-132/172.16.0.132
Start Time:   Sat, 13 Feb 2021 18:41:23 +0000
Labels:       app=churn
              pod-template-hash=6955975fff
              version=v1
Annotations:  cni.projectcalico.org/podIP: 10.1.124.78/32
              cni.projectcalico.org/podIPs: 10.1.124.78/32
              sidecar.istio.io/inject: true
Status:       Running
IP:           10.1.124.78
IPs:
  IP:           10.1.124.78
Controlled By:  ReplicaSet/tfserving-6955975fff
Containers:
  churn-container:
    Container ID:  containerd://4be43fddef618c11dc2043ec9e7d51de33ef8efd44d10d096dc43e6fb7beb88b
    Image:         ugoiloh/mavencode:1.0.2
    Image ID:      docker.io/ugoiloh/mavencode@sha256:0bf36e03d28195aea4dcdaeaf9ae8bcaa3d34604374a3a0e3aa74dab8755234d
    Ports:         9000/TCP, 8500/TCP
    Host Ports:    0/TCP, 0/TCP
    Command:
      /usr/bin/tensorflow_model_server
    Args:
      --port=9000
      --

In [98]:
!kubectl get svc tfserving-service -n tfserving

NAME                TYPE           CLUSTER-IP       EXTERNAL-IP   PORT(S)                         AGE
tfserving-service   LoadBalancer   10.152.183.175   <pending>     9000:31358/TCP,8500:32328/TCP   8m24s


In [99]:
!kubectl logs -c churn-container tfserving-6955975fff-hn9kl -n tfserving

2021-02-13 18:41:25.103571: I tensorflow_serving/model_servers/server.cc:88] Building single TensorFlow model file config:  model_name: churn model_base_path: /models
2021-02-13 18:41:25.103817: I tensorflow_serving/model_servers/server_core.cc:464] Adding/updating models.
2021-02-13 18:41:25.103839: I tensorflow_serving/model_servers/server_core.cc:587]  (Re-)adding model: churn
2021-02-13 18:41:25.204356: I tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: churn version: 1}
2021-02-13 18:41:25.204403: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: churn version: 1}
2021-02-13 18:41:25.204415: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: churn version: 1}
2021-02-13 18:41:25.204468: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:32] Reading SavedModel from: /models/1
2021-02-13 18:41:25.206814: I external/org_tensorflow/tensorflow/cc/saved_m

In [ ]:
!kubectl port-forward tfserving-6955975fff-hn9kl 8500:8500 -n tfserving

In [132]:
!curl -i -v -d @input.json -X POST http://localhost:8500/v1/models/churn:predict

*   Trying ::1...
* TCP_NODELAY set
* Connected to localhost (::1) port 8500 (#0)
> POST /v1/models/churn:predict HTTP/1.1
> Host: localhost:8500
> User-Agent: curl/7.58.0
> Accept: */*
> Content-Length: 281
> Content-Type: application/x-www-form-urlencoded
> 
* upload completely sent off: 281 out of 281 bytes
< HTTP/1.1 200 OK
HTTP/1.1 200 OK
< Content-Type: application/json
Content-Type: application/json
< Date: Sun, 14 Feb 2021 04:04:45 GMT
Date: Sun, 14 Feb 2021 04:04:45 GMT
< Content-Length: 90
Content-Length: 90

< 
{
    "predictions": [[0.0167271495], [0.165913582], [0.0756933689], [0.595268786]
    ]
* Connection #0 to host localhost left intact
}

In [116]:
input_data = {
    "instances": [
        [0.672, 32, 6, 0.0, 2, 1, 1, 0.895494, 1, 0, 0, 0, 1],
        [0.564, 42, 4, 0.47686, 2, 1, 1, 0.979930, 0, 1, 0, 0, 1],
        [0.418, 24, 3, 0.457317, 1, 1, 0, 0.429438, 0, 0, 1, 0, 1],
        [0.760, 47, 7, 0.0, 1, 1, 0, 0.166824, 1, 0, 0, 1, 0]
    ]
}